# Install dependencies

In [ ]:
#installing dependencies
!pip install -U --pre triton
!pip install -U xformers
!pip install accelerate transformers datasets
!pip install bitsandbytes
!pip install diffusers
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install peft
!pip install tensorboard
!pip install safetensors

In [ ]:
#installing the latest version of diffusers to mitigate errors
!pip uninstall -y diffusers
!pip install git+https://github.com/huggingface/diffusers.git

In [ ]:
#upgrading the following libraries to mitigate errors
!pip install --upgrade transformers accelerate 

In [ ]:
#logging into huggingface hub
from huggingface_hub import notebook_login
notebook_login()

# Unzip Image Dataset


In [ ]:
# unzip tool
!apt-get update
!apt-get install -y unzip

In [ ]:
# unzip image dataset
# wait till the upload is complete
!unzip /workspace/pytorch_lora_weights.safetensors.zip -d /workspace/

Archive:  /workspace/pytorch_lora_weights.safetensors.zip
  inflating: /workspace/pytorch_lora_weights.safetensors  
  inflating: /workspace/__MACOSX/._pytorch_lora_weights.safetensors  


In [ ]:
# terminate  
!ps aux | grep accelerate

In [ ]:
# remove saved checkpoint directory
!rm -rf /workspace/kanji-diffusion-v1-4

In [ ]:
# This is the main training script
# this pass the parameter to train_text_image_lora.py 
!accelerate launch --mixed_precision="no" --num_processes=1 --num_machines=1 --dynamo_backend="no" train_text_to_image_lora.py \
    --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4" \
    --train_data_dir="/workspace/traindata" \
    --image_column="image" \
    --caption_column="text" \
    --resolution=128 \
    --lora_rank=32 \
    --train_batch_size=1 \
    --num_train_epochs=8 \
    --checkpointing_step=2560 \
    --validation_steps=2560 \
    --learning_rate= \
    --lr_scheduler="cosine" \
    --lr_warmup_steps=0 \
    --seed=42 \
    --output_dir="kanji-diffusion-v1-4" \
    --report_to="tensorboard" \
    --logging_dir="logs"

# Test Inference Output

In [ ]:
#Loading the pre-trained Stable Diffusion model and moving it to GPU for efficient processing.
from diffusers import StableDiffusionPipeline
import torch

model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# Load and apply LoRA weights
lora_model_path = "/workspace/pytorch_lora_weights.safetensors"
pipe.unet.load_attn_procs(lora_model_path)

In [ ]:
# Defines a function to generate and display images based on text prompts
def generate_image(prompt, num_images=1):
    images = pipe(prompt, num_images_per_prompt=num_images,height=128,width=128).images
    for i, image in enumerate(images):
        display(image)
        image.save(f"generated_image_{i}.png")

In [ ]:
# mount sample prompts
sample_prompts = [
    "dog",
    "cat",
    "car",
    "school",
    "pizza",
    "train",
    "ball",
    "star",
    "milk",
    "home"
]

In [ ]:
import os

# Folder to save images
out_dir = "output-american-culture"
os.makedirs(
    out_dir, exist_ok=True)

# Loop over your prompts
for prompt in cultural_simple:
    images = pipe(prompt, num_images_per_prompt=20, height=128, width=128).images
    for i, image in enumerate(images, start=1):
        filename = f"{prompt.replace(' ', '_')}{i}.png"
        save_path = os.path.join(out_dir, filename)
        image.save(save_path)
        display(image)  # optional, shows in notebook
        print(f"✅ Saved {save_path}")